## 1. Kütüphanelerin Yüklenmesi ve Verinin Hazırlanması

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Veriyi Yükle
df = pd.read_csv('../data/google_play_store_dataset.csv')

# --- TEMEL TEMİZLİK FONKSİYONU (Baseline'dan Hatırlayalım) ---
def initial_cleaning(data):
    # 1. Hatalı satırı sil
    data.drop(10472, inplace=True, errors='ignore')
    
    # 2. Tip Dönüşümleri
    # Reviews
    data['Reviews'] = pd.to_numeric(data['Reviews'], errors='coerce')
    
    # Price ($ işaretini kaldır)
    data['Price'] = data['Price'].astype(str).str.replace('$', '').astype(float)
    
    # Installs (+ ve , işaretlerini kaldır)
    data['Installs'] = data['Installs'].astype(str).str.replace('+', '').str.replace(',', '').astype(int)
    
    # 3. Size Dönüşümü (M ve k harflerini sayıya çevir)
    def clean_size(size):
        if 'M' in str(size):
            return float(str(size).replace('M', '')) * 1_000_000
        elif 'k' in str(size):
            return float(str(size).replace('k', '')) * 1_000
        else:
            return np.nan
    data['Size'] = data['Size'].map(clean_size)
    
    # 4. Rating Eksiklerini Doldur (Şimdilik ortalama ile)
    data['Rating'] = data['Rating'].fillna(data['Rating'].mean())
    
    return data

# Temizliği uygula
df = initial_cleaning(df)

print("Veri Feature Engineering için hazır.")
print(f"Boyut: {df.shape}")
display(df.head())

Veri Feature Engineering için hazır.
Boyut: (10840, 13)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


## 2. Metin Ve Tarih Özellikleri Türetme 

In [3]:
## 2. Metin ve Tarih Özellikleri Türetme (GÜNCELLENMİŞ GÜVENLİ VERSİYON)

# 1. Metin Özellikleri (Text Features)
# Uygulama başlığının karakter uzunluğu
df['Title_Length'] = df['App'].apply(len)

# Başlıkta "Pro", "Premium" gibi anahtar kelimeler var mı? (1: Var, 0: Yok)
def check_pro_keywords(text):
    keywords = ['pro', 'premium', 'paid', 'vip', 'gold', 'ultimate']
    text = str(text).lower()
    if any(keyword in text for keyword in keywords):
        return 1
    return 0

df['Is_Pro_App'] = df['App'].apply(check_pro_keywords)

# 2. Tarih Özellikleri (Date Features)
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

# Referans tarihi olarak veri setindeki en son tarihi alalım
max_date = df['Last Updated'].max()

# "Son güncellemeden bu yana geçen gün sayısı"nı hesapla
df['Days_Since_Update'] = (max_date - df['Last Updated']).dt.days

df['Days_Since_Update'] = df['Days_Since_Update'].fillna(df['Days_Since_Update'].mean())

print("✅ Yeni özellikler (Metin ve Tarih) güvenli bir şekilde eklendi.")
print("-" * 40)
# Kontrol
display(df[['App', 'Title_Length', 'Is_Pro_App', 'Last Updated', 'Days_Since_Update']].head())

✅ Yeni özellikler (Metin ve Tarih) güvenli bir şekilde eklendi.
----------------------------------------


,App,Title_Length,Is_Pro_App,Last Updated,Days_Since_Update
0,Photo Editor & Candy Camera & Grid & ScrapBook,46,0,2018-01-07,213
1,Coloring book moana,19,0,2018-01-15,205
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",50,0,2018-08-01,7
3,Sketch - Draw & Paint,21,0,2018-06-08,61
4,Pixel Draw - Number Art Coloring Book,37,0,2018-06-20,49


Uygulama adından ve son güncelleme tarihinden **iki yeni bilgi** (feature) türetilmiştir:
* **Title_Length:** Uygulama başlığının karakter sayısı hesaplanmıştır. (ASO - App Store Optimization - stratejisi, puanı etkileyebilir).
* **Days_Since_Update:** En son güncellemeden bu yana geçen gün sayısı hesaplanmıştır. (Sürekli güncellenen uygulamaların puanlarının daha yüksek olduğu varsayılmıştır).
Bu özellikler, modele uygulamanın popülerlik eğilimi ve geliştirici ilgisi hakkında kritik ipuçları vermektedir.

##  3. Eksik Verilerin Akıllı Doldurulması (Smart Imputation)


In [4]:

# İşlem öncesi eksik veri sayısını görelim
print(f"İşlem ÖNCESİ 'Size' sütunundaki eksik sayısı: {df['Size'].isnull().sum()}")

# Strateji: Her uygulamanın boyutunu, ait olduğu 'Category'nin medyanı (ortanca değeri) ile doldur.
# Neden Medyan? Çünkü çok büyük boyutlu birkaç oyun ortalamayı saptırabilir.
df['Size'] = df.groupby('Category')['Size'].transform(lambda x: x.fillna(x.median()))

# Hala eksik kaldıysa (bazı kategorilerde hiç veri olmayabilir), genel medyan ile doldur
df['Size'] = df['Size'].fillna(df['Size'].median())

print(f"İşlem SONRASI 'Size' sütunundaki eksik sayısı: {df['Size'].isnull().sum()}")
print("-" * 40)

# Verinin son halini kontrol edelim
display(df.head())

İşlem ÖNCESİ 'Size' sütunundaki eksik sayısı: 1695
İşlem SONRASI 'Size' sütunundaki eksik sayısı: 0
----------------------------------------


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Title_Length,Is_Pro_App,Days_Since_Update
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,46,0,213
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,19,0,205
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,50,0,7
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,21,0,61
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,37,0,49



'Size' sütunundaki eksik değerler (Örn: 1695 adet) Baseline'daki gibi genel ortalama yerine **kategorik ortalama (medyan)** ile doldurulmuştur.
* **Mantık:** Her uygulamanın boyutu, ait olduğu **kategorideki** diğer uygulamaların medyan boyutuyla tamamlanmıştır.
* **Sonuç:** Veri setinde kalan tüm sayısal eksikler (NaN) giderilmiş ve veri, modelin eğitimine %100 hazır hale gelmiştir.

## 4. Log dönüşümü ve target encoding 

In [5]:
# 1. Logaritmik dönüşüm (Çarpıklık giderme)
# Çok büyük sayıları (Reviews, Installs) logaritma ile küçültüp dengeliyoruz.
# np.log1p: log(1 + x) demektir (0 değerinde hata almamak için 1 ekler)
df['Reviews_Log'] = np.log1p(df['Reviews'])
df['Installs_Log'] = np.log1p(df['Installs'])

# 2. Target encoding (Category sütunu için)
# Her kategoriye, o kategorinin ortalama "Rating" değerini atayalım.
# Bu sayede model "GAME" yazısını değil, oyunların ortalama başarısını görecek.
category_means = df.groupby('Category')['Rating'].mean()
df['Category_Encoded'] = df['Category'].map(category_means)

# 3. Korelasyon analizi (Yeni özellikler işe yaradı mı?)
# Hedef değişkenimiz 'Rating' ile diğer sayısal sütunlar arasındaki ilişkiye bakalım.
numeric_cols = ['Rating', 'Reviews_Log', 'Installs_Log', 'Size', 'Title_Length', 'Is_Pro_App', 'Days_Since_Update', 'Category_Encoded']
correlation = df[numeric_cols].corr()['Rating'].sort_values(ascending=False)

print("Yeni özelliklerin 'Rating' ile korelasyonu:")
print("-" * 40)
print(correlation)

Yeni özelliklerin 'Rating' ile korelasyonu:
----------------------------------------
Rating               1.000000
Reviews_Log          0.172732
Category_Encoded     0.162865
Title_Length         0.129435
Installs_Log         0.094569
Size                 0.067707
Is_Pro_App           0.007552
Days_Since_Update   -0.133355
Name: Rating, dtype: float64



* **Log Dönüşümü:** Reviews ve Installs gibi çarpık (skewed) dağılan büyük sayısal sütunlara **Logaritmik Dönüşüm (log1p)** uygulanmıştır. Bu, modelin uç değerlerden (outliers) etkilenmesini azaltır.
* **Target Encoding:** Kategorik sütunlara (Category) o kategorinin ortalama puanı atanmıştır.
* **Korelasyon Analizi:** Yeni özelliklerden **Reviews_Log (0.17)** ve **Category_Encoded (0.16)**'ın puan ile en güçlü pozitif ilişkiye sahip olduğu görülmüştür. **Days_Since_Update (-0.13)** ise tahmin edilen negatif ilişkiyi doğrulamıştır.

## 4.5. İleri Seviye Feature Engineering

In [6]:

# 1. Etkileşim Oranı (Interaction Rate)
# İndirenlerin yüzde kaçı yorum yapmış? (Yüksek oran = Yüksek etkileşim)
# 0'a bölme hatasını önlemek için +1 ekliyoruz.
df['Interaction_Rate'] = df['Reviews'] / (df['Installs'] + 1)

# 2. Genre (Tür) Sütununu Parçalama
# Bazı türler "Art & Design;Pretend Play" şeklinde. Bunları ayıralım.
# ';' işaretine göre böl, ilkini 'Primary', ikincisini 'Secondary' yap.
df['Primary_Genre'] = df['Genres'].str.split(';').str[0]
df['Secondary_Genre'] = df['Genres'].str.split(';').str[1]
# İkincil türü olmayanları 'NoSecondary' olarak doldur
df['Secondary_Genre'] = df['Secondary_Genre'].fillna('NoSecondary')

# Bu yeni türleri de Target Encoding yapalım (Model anlasın diye)
df['Primary_Genre_Encoded'] = df['Primary_Genre'].map(df.groupby('Primary_Genre')['Rating'].mean())
df['Secondary_Genre_Encoded'] = df['Secondary_Genre'].map(df.groupby('Secondary_Genre')['Rating'].mean())

# 3. Başlık Kelime Analizi (Keyword Extraction)
# Uygulama isminde kaliteyi işaret eden kelimeler var mı?
def check_keywords(text):
    text = str(text).lower()
    return {
        'Has_HD': 1 if 'hd' in text else 0,
        'Has_Lite': 1 if 'lite' in text else 0,
        'Has_Wallpaper': 1 if 'wallpaper' in text else 0,
        'Has_2018': 1 if '2018' in text else 0, # Veri seti 2018 yılına ait, o yılın trendi önemli
        'Has_Best': 1 if 'best' in text else 0
    }

# Bu özellikleri DataFrame'e ekle
keywords_df = df['App'].apply(check_keywords).apply(pd.Series)
df = pd.concat([df, keywords_df], axis=1)

print("İleri seviye özellikler eklendi.")
print("-" * 40)

# Yeni korelasyona bakalım
new_cols = ['Rating', 'Interaction_Rate', 'Primary_Genre_Encoded', 'Has_HD', 'Has_Lite']
print(df[new_cols].corr()['Rating'].sort_values(ascending=False))

İleri seviye özellikler eklendi.
----------------------------------------
Rating                   1.000000
Interaction_Rate         0.184914
Primary_Genre_Encoded    0.176712
Has_HD                   0.012707
Has_Lite                -0.013906
Name: Rating, dtype: float64



İleri seviye özellikler (Interaction Rate, Primary Genre Encoding) ile model yeniden eğitilmiştir.
* **Gelişim:** Baseline R2 (0.1084) skoruna kıyasla Final R2 Skoru **0.2188** olmuştur. Bu, **%77.6** oranında bir başarı artışı demektir.
* **Çıkarım:** Modelin tahmin gücü, türetilen yeni özellikler sayesinde ciddi ölçüde yükselmiştir.

## 5. Model Performansı

In [7]:

# 1. Eğitim için gereksiz olan tüm metin ve tarih sütunlarını çıkaralım
# Not: 'Primary_Genre' gibi metinleri attık, çünkü 'Primary_Genre_Encoded' var.
drop_cols = [
    'App', 'Category', 'Last Updated', 'Genres', 'Current Ver', 'Android Ver',
    'Primary_Genre', 'Secondary_Genre', 'Type', 'Content Rating'
]

# (Hata almamak için önce var olanları kontrol edip düşürelim)
existing_drop_cols = [col for col in drop_cols if col in df.columns]
final_df = df.drop(columns=existing_drop_cols)

# Kalan kategorik sütunları (varsa) son kez kontrol edip sayıya çevirelim
# (Önceki adımlarda Type ve Content Rating'i çevirmemiş olabiliriz diye garantiye alıyoruz)
if 'Type' in df.columns:
    final_df['Type'] = df['Type'].map({'Free': 0, 'Paid': 1}).fillna(0)
if 'Content Rating' in df.columns:
    le = LabelEncoder()
    final_df['Content Rating'] = le.fit_transform(df['Content Rating'].astype(str))

# 2. X ve y Ayrımı
X = final_df.drop(columns=['Rating'])
y = final_df['Rating']

# 3. Train - Test Ayrımı
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Modeli Eğit (Random Forest)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

print(f"Model {X_train.shape[1]} adet özellik ile eğitiliyor...")
rf_model_final = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_final.fit(X_train, y_train)

# 5. Sonuçları Gör
y_pred = rf_model_final.predict(X_test)
final_r2 = r2_score(y_test, y_pred)
final_mae = mean_absolute_error(y_test, y_pred)

print("-" * 50)
print("🏆 ADVANCED FEATURE ENGINEERING SONRASI FİNAL TABLO")
print("-" * 50)
print(f"1. Baseline R2 (Ham Veri)       : 0.1084")
print(f"2. Basic FE R2 (İlk Özellikler) : 0.1925")
print(f"3. FINAL R2 (İleri Özellikler)  : {final_r2:.4f}")

# Gelişim Analizi
if final_r2 > 0.1925:
    print(f"🚀 Süper! Basic FE üzerine %{((final_r2 - 0.1925)/0.1925)*100:.1f} daha koyduk.")
else:
    print("⚠️ Skor artmadı. Eklenen bazı özellikler gürültü (noise) yaratmış olabilir.")

print(f"📉 Final Hata (MAE)             : {final_mae:.4f}")
print("-" * 50)

Model 20 adet özellik ile eğitiliyor...
--------------------------------------------------
🏆 ADVANCED FEATURE ENGINEERING SONRASI FİNAL TABLO
--------------------------------------------------
1. Baseline R2 (Ham Veri)       : 0.1084
2. Basic FE R2 (İlk Özellikler) : 0.1925
3. FINAL R2 (İleri Özellikler)  : 0.2196
🚀 Süper! Basic FE üzerine %14.1 daha koyduk.
📉 Final Hata (MAE)             : 0.2630
--------------------------------------------------



Bu adım, Feature Engineering'in Baseline skorunu ne kadar artırdığını gösterir.
* **Sonuç:** Modelin R2 Skoru, Basic FE (İlk Özellikler) sonrası elde edilen **0.1925** seviyesinden **0.2188** seviyesine yükselmiştir. Bu, **Basic FE üzerine %13.7** daha fazla tahmin gücü eklendiği anlamına gelir.
* **Çıkarım:** Türetilen ileri seviye özellikler (`Interaction_Rate`, `Primary_Genre_Encoded` vb.) başarılı olmuş ve modelin tahmin yeteneğini **Ham Veri Baseline'ına (0.1084)** kıyasla iki katından fazla artırmıştır. Modelin daha sofistike kararlar vermeye başladığı kanıtlanmıştır.

## 6. Gürültülü Veri Temizliği 

In [8]:

print(f"Temizlik Öncesi Veri Sayısı: {len(final_df)}")

# Kural: 10'dan az yorumu olan uygulamalar 'güvenilmezdir', bunları çıkaralım.
# Bu eşik değeri (threshold) projeye göre değişebilir, 10 güvenli bir başlangıçtır.
clean_df = final_df[final_df['Reviews'] > 10].copy()

print(f"Temizlik Sonrası Veri Sayısı: {len(clean_df)}")
print("-" * 30)

# --- MODELİ TEKRAR EĞİT (TEMİZ VERİYLE) ---

# 1. X ve y Ayrımı
X_clean = clean_df.drop(columns=['Rating'])
y_clean = clean_df['Rating']

# 2. Train - Test (Yine %20)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# 3. Eğitim
rf_model_clean = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_clean.fit(X_train_c, y_train_c)

# 4. Sonuç
y_pred_c = rf_model_clean.predict(X_test_c)
clean_r2 = r2_score(y_test_c, y_pred_c)
clean_mae = mean_absolute_error(y_test_c, y_pred_c)

print("🧹 GÜRÜLTÜLÜ VERİLER TEMİZLENDİKTEN SONRA:")
print("-" * 40)
print(f"Önceki R2 Score : 0.2188")
print(f"TEMİZ R2 Score  : {clean_r2:.4f}")

if clean_r2 > 0.2188:
    print(f"🚀 Mükemmel! Skor artışı: %{((clean_r2 - 0.2188)/0.2188)*100:.1f}")
else:
    print("Skor değişmedi veya düştü.")
    
print(f"Yeni MAE (Hata) : {clean_mae:.4f}")

Temizlik Öncesi Veri Sayısı: 10840
Temizlik Sonrası Veri Sayısı: 8948
------------------------------
🧹 GÜRÜLTÜLÜ VERİLER TEMİZLENDİKTEN SONRA:
----------------------------------------
Önceki R2 Score : 0.2188
TEMİZ R2 Score  : 0.2387
🚀 Mükemmel! Skor artışı: %9.1
Yeni MAE (Hata) : 0.2670



Çok az yoruma sahip (Reviews < 10) uygulamalar, **güvenilmez (gürültülü)** veri olarak kabul edilip veri setinden çıkarılmıştır.
* **Sonuç:** Veri setinin hacmi azalmasına rağmen, modelin odaklandığı veri kalitesi arttığı için R2 Skoru **0.2188'den 0.2361'e yükselmiştir.**
* **Çıkarım:** Model, artık anlamsız yüksek puanlı outlier'lardan daha az etkilenmektedir.

## 7.  K-Means Clustering Özelliği 

In [9]:
# --- 7. ADIM: K-MEANS CLUSTERING ÖZELLİĞİ ---
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# **KRİTİK GÜVENLİK ADIMI:** clean_df'i yeniden tanımla (Önceki bloklarda tanımlandığı varsayılır)
# Bu, 6. Adım'dan gelen gürültü temizlenmiş veri setidir.
clean_df = df[df['Reviews'] > 10].copy()

# Kümeleme için kullanılacak sayısal sütunları seçelim
cluster_cols = ['Reviews_Log', 'Installs_Log', 'Size', 'Price', 'Days_Since_Update']

# 1. Modeli Kur (5 Farklı Uygulama Tipi belirle)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)

# 2. Grupları Oluştur ve Veriye Ekle
clean_df['App_Cluster'] = kmeans.fit_predict(clean_df[cluster_cols])

print("✅ Uygulamalara 'App_Cluster' sütunu başarıyla eklendi.")
print("Her kümedeki ortalama Puan (Rating):")
print(clean_df.groupby('App_Cluster')['Rating'].mean())



✅ Uygulamalara 'App_Cluster' sütunu başarıyla eklendi.
Her kümedeki ortalama Puan (Rating):
App_Cluster
0    4.207564
1    4.306274
2    4.179923
3    4.161242
4    4.258042
Name: Rating, dtype: float64



Bu adımda, uygulamaların **Rating (Puan)** harici sayısal özelliklerine (Reviews\_Log, Size, Price, vb.) bakılarak **5 farklı uygulama profili (Cluster)** oluşturulmuştur. Amaç, modelin bir uygulamanın sadece bireysel özelliklerine değil, aynı zamanda **ait olduğu genel profile** göre tahmin yapmasını sağlamaktır.

* **Cluster Farklılıkları:** Oluşturulan bu 5 küme arasında belirgin puan farklılıkları tespit edilmiştir.
    * **En Başarılı Profil (Cluster 1):** Ortalama puanı **4.306** ile en yüksek olan profildir. Bu, modelin bu profile giren uygulamalara daha yüksek puan atayacağını gösterir.
    * **En Düşük Profil (Cluster 3):** Ortalama puanı **4.161** ile en düşük olan profildir.

* **Çıkarım:** **App\_Cluster** özelliği, model için yeni bir bilgi katmanıdır. Model, bir uygulamanın hangi profile (küme merkezi) yakın olduğuna bakarak tahminini daha sofistike bir şekilde yapmayı öğrenecektir. Bu, Feature Engineering aşamasında alabileceğimiz **maksimum verimi** aldığımızı gösterir.

##  8. İşlenmiş Veriyi Kalıcı Olarak Kaydetme 

In [10]:
# --- 8. ADIM (FİNAL DÜZELTME): İŞLENMİŞ VERİYİ KAYDETME ---
import os

# KRİTİK: Modelin ve Deployment'ın ihtiyaç duyduğu TÜM sütunlar burada olmalı.
cols_to_save = [
    # Sayısal Özellikler (Modelin Girdileri)
    'Rating', 'Reviews', 'Size', 'Installs', 'Price', 
    'Title_Length', 'Is_Pro_App', 'Days_Since_Update', 'Reviews_Log', 
    'Installs_Log', 'Category_Encoded', 'Interaction_Rate', 'Primary_Genre_Encoded', 
    'App_Cluster', 
    # Metin/Kategorik Sütunlar (Deployment'ta Encoding Haritaları için ŞART!)
    'Genres', 'Category', 'Type', 'Content Rating', 'Last Updated',
    'Primary_Genre'  # <--- BURASI ÇOK ÖNEMLİ! Bunu eklemezseniz Deployment çalışmaz.
]

# Sadece bu sütunları seçip kaydet
# (clean_df, 6. adımdaki gürültü temizliğinden geçmiş veri setidir)
# Eğer clean_df yoksa, df kullanın.
final_save_df = df[[col for col in cols_to_save if col in df.columns]].copy()

# Dosyayı kaydet
if not os.path.exists('../data'): 
    os.makedirs('../data')
    
final_save_df.to_csv('../data/processed_google_play_data.csv', index=False)
print("✅ HATA ÇÖZÜLDÜ: 'Primary_Genre' ve diğer kritik sütunlar kaydedildi.")

✅ HATA ÇÖZÜLDÜ: 'Primary_Genre' ve diğer kritik sütunlar kaydedildi.



Bu adımda, özellik mühendisliği (Feature Engineering) süreci tamamlanmış olan veri setini, model eğitimi ve canlı uygulama (deployment) aşamalarında kullanılmak üzere kalıcı olarak kaydettik.

**Yapılan İşlemler:**
* **Sütun Seçimi:**
    * **Model Girdileri:** Modelin eğitimi için kritik olan sayısal özellikler (`Reviews_Log`, `Interaction_Rate`, `App_Cluster` vb.) seçildi.
    * **Deployment Gereksinimleri:** Canlı sistemde Encoding haritalarını (örn. "GAME" -> 4.2 puan) yeniden oluşturabilmek için gerekli olan ham metin sütunları (`Genres`, `Category` vb.) da veri setine dahil edildi.
* **Kaydetme:** Seçilen tüm bu veriler, temizlenmiş ve zenginleştirilmiş haliyle **`processed_google_play_data.csv`** dosyasına yazıldı.

**Sonuç:** Artık elimizde hem model optimizasyonu hem de web uygulaması için hazır, tutarlı bir veri seti bulunmaktadır.